In [3]:
# Imports needed across sections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from tabulate import tabulate
from IPython.display import display, HTML

In [ ]:
print("The shape of the departure data is ", departure_data.shape)
print("Departure Data Info:")
print(departure_data.info())
print("\nFirst few rows of Departure Data:")
print(departure_data.head())

print("\nSummary Statistics for Departure Data:")
print(departure_data.describe())

In [ ]:
# More visually appealing heads of the data

departure_head_html = tabulate(departure_data.head(), headers='keys', tablefmt='html')
passengers_head_html = tabulate(passengers_data.head(), headers='keys', tablefmt='html')

departure_table_html = f'<h3>Departures Data</h3>{departure_head_html}'
passengers_table_html = f'<h3>Passengers Data</h3>{passengers_head_html}'

display(HTML(departure_table_html))
display(HTML(passengers_table_html))

In [ ]:
# Create top 10 busiest airports barplot.

airport_passengers = passengers_data.groupby('usg_apt')['Total'].sum().reset_index()
top_10_airports = airport_passengers.sort_values(by='Total', ascending=False).head(10)

# Create plot
plt.figure(figsize=(12, 6))
plt.bar(top_10_airports['usg_apt'], top_10_airports['Total'])
plt.xlabel('Airport')
plt.ylabel('Total Passengers')
plt.title('Top 10 Busiest Airports by Total Passengers')
plt.tight_layout()
plt.show()

In [ ]:
# Yearly Variation in the Number of Flights
yearly_flights = departure_data.groupby('Year')['Total'].sum()

plt.figure(figsize=(10, 6))
plt.scatter(yearly_flights.index, yearly_flights.values, color='skyblue', label='Yearly Flights', marker='o')

# Make line plot
plt.plot(yearly_flights.index, yearly_flights.values, color='orange', linestyle='-', marker='', label='Trend Line')

plt.xlabel('Year')
plt.ylabel('Total Number of Flights')
plt.title('Yearly Variation in the Number of Flights')
plt.xticks(yearly_flights.index, rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
yearly_passengers = passengers_data.groupby('Year')['Total'].sum()

# Percent change from the previous year
percent_change = yearly_passengers.pct_change() * 100

plt.figure(figsize=(10, 6))
plt.bar(percent_change.index, percent_change.values, color='skyblue')
plt.xticks(percent_change.index, rotation=45, ha='right')

plt.xlabel('Year')
plt.ylabel('Percent Change')
plt.title('Percent Change of Passengers per Year')
plt.xticks(percent_change.index)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
yearly_departures = departure_data.groupby('Year')['Total'].sum()

# Group the passengers data by year, calculate the total number of passengers for each year
yearly_passengers = passengers_data.groupby('Year')['Total'].sum()

# Create a scatter plot to visualize the correlation between departures and passengers
plt.figure(figsize=(10, 6))
plt.scatter(yearly_departures, yearly_passengers, color='skyblue', alpha=0.7)
plt.xlabel('Total Departures')
plt.ylabel('Total Passengers')
plt.title('Correlation between Departures and Passengers')
plt.grid()

correlation_coefficient = yearly_departures.corr(yearly_passengers)

# Display correlation coefficient
plt.text(300000, 3e7, f'Correlation Coefficient: {correlation_coefficient:.2f}', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Sort the data by passenger traffic and select the top 20 busiest airports
top_20_airports = airport_passengers.sort_values(by='Total', ascending=False).head(20)

print(top_20_airports)

# Create a map
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add markers to the map
for index, row in top_20_airports.iterrows():
    usg_apt = row['usg_apt']
    airport_info = airport_data[airport_data['AIRPORT'] == usg_apt]

    if not airport_info.empty:
        airport_info = airport_info.iloc[0]
        airport_name = airport_info['DISPLAY_AIRPORT_NAME']
        latitude = airport_info['LAT_DEGREES']
        longitude = airport_info['LONGITUDE']
        if not pd.isna(latitude) and not pd.isna(longitude):
            folium.Marker([latitude, longitude], tooltip=airport_name).add_to(m)

m.save('top_20_airports_map.html')
m

In [ ]:
# Plotting graphs representing female vs male results in each category

plt.figure(1, figsize=(16,10))
sns.pairplot(data=df, hue='gender')
plt.show()